In [2]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np

cuda:0


In [3]:
import torch
import torch.nn as nn

# 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu2 = nn.ReLU()

        # 使用Xavier初始化来初始化权重
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)

        # 初始化偏置为零
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        return out
        
# 定义输入、隐藏层和输出的维度
input_size = 10
hidden_size = 20
output_size = 10

# 创建一个SimpleNN模型



In [4]:
class customloss(nn.Module):
    def __init__(self, theta):
        super(customloss, self).__init__()
        self.theta = theta
    def forward(self, output_list, y_list):
        theta = torch.tensor(self.theta, dtype= torch.float32).to(device) 
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)))  
            loss += torch.sum(-y * torch.log(prob)) 
            index += 1  
        return loss / (len(y_list)/100)


In [5]:
CONTEXT_ARRAY = np.load('features_uni.npy')
theta = np.load('theta_gau.npy')
PURCHASE_LIST = np.load('purchase_uni.npy')

In [6]:
feature_list = []
purchase_list = []

model = SimpleNN(input_size, hidden_size, output_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)
for t in range(0,1000):
    context = CONTEXT_ARRAY[t]

    purchase_list.append(PURCHASE_LIST[t])

    feature_list.append(np.array(context))

    if t%10 == 1:
        loss_function = customloss(theta)
        epochs = 1
        for epoch in range(epochs):
            
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in feature_list]
            loss = loss_function(output_list, purchase_list)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 1727.91552734375
epoch 0, loss 1694.7509765625
epoch 0, loss 1682.581787109375
epoch 0, loss 1674.61474609375
epoch 0, loss 1671.0302734375
epoch 0, loss 1667.1309814453125
epoch 0, loss 1664.7103271484375
epoch 0, loss 1663.7701416015625
epoch 0, loss 1663.67919921875
epoch 0, loss 1663.6793212890625
epoch 0, loss 1663.73486328125
epoch 0, loss 1663.7315673828125
epoch 0, loss 1663.6158447265625
epoch 0, loss 1663.561767578125
epoch 0, loss 1663.5616455078125
epoch 0, loss 1663.5625
epoch 0, loss 1663.56201171875
epoch 0, loss 1663.5615234375
epoch 0, loss 1663.561279296875
epoch 0, loss 1663.5606689453125
epoch 0, loss 1663.559814453125
epoch 0, loss 1663.5595703125
epoch 0, loss 1663.558349609375
epoch 0, loss 1663.5567626953125
epoch 0, loss 1663.555908203125
epoch 0, loss 1663.5556640625
epoch 0, loss 1663.5560302734375
epoch 0, loss 1663.5560302734375
epoch 0, loss 1663.556396484375
epoch 0, loss 1663.556640625
epoch 0, loss 1663.5567626953125
epoch 0, loss 1663.556

In [7]:
problist  = np.load('prob_list.npy')
def hhh(i):
    ff = model(torch.tensor(CONTEXT_ARRAY[i],dtype = torch.float32).to(device))
    ff = ff.cpu().detach().numpy()
    u = (ff @ theta.T ).reshape(1,-1)
    prob = np.exp(u) / (np.sum(np.exp(u)))
    
    print(problist[i])
    print(prob)

hhh(100)   

[1.47919351e-01 4.88826777e-04 8.65607633e-03 3.62747532e-04
 5.65246786e-01 2.92663029e-03 2.73947878e-01 4.51703708e-04]
[[0.12503353 0.12492588 0.12497929 0.12507289 0.12505956 0.12495974
  0.12504535 0.12492376]]
